In [2]:
import itk
import itkwidgets as itkw

image_itk = itk.imread('./cT1wNeuro.nrrd')
castImageFilter = itk.CastImageFilter[image_itk, itk.Image[itk.F,3]].New(image_itk)
castImageFilter.Update()
image_itk_float = castImageFilter.GetOutput()

## Gradient Anisotropic Diffusion Filter

In [ ]:
grad_ani_dif_filter = itk.GradientAnisotropicDiffusionImageFilter[image_itk_float, itk.Image[itk.F,3]].New(image_itk_float)
grad_ani_dif_filter.Update()

smooth_gadf = grad_ani_dif_filter.GetOutput()
itkw.view(smooth_gadf, cmap='Grayscale', mode='x')

## Apply a recursive gaussian filter which approximates to an IIR (infinite impulse response) convolution with the Gaussian kernel.

In [8]:
recursive_gaussian_filter = itk.RecursiveGaussianImageFilter[image_itk_float, itk.Image[itk.F,3]].New(image_itk_float)
recursive_gaussian_filter.SetSigma(2)
recursive_gaussian_filter.SetZeroOrder()
recursive_gaussian_filter.Update()

smooth_rgf = recursive_gaussian_filter.GetOutput()
itkw.view(smooth_rgf, cmap='Grayscale', mode='x')

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…

### Interactive Recursive Gaussian

In [3]:
from ipywidgets import interactive
import ipywidgets as widgets
viewer = itkw.view(image_itk_float, cmap='Grayscale', mode='x', annotations=False)

# Create an itk smoother filter object. By re-using the object, output memory-reallocation is avoided
recursive_gaussian_filter = itk.RecursiveGaussianImageFilter[image_itk_float, itk.Image[itk.F,3]].New(image_itk_float)
recursive_gaussian_filter.SetZeroOrder()

def recgauss_and_view(sigma=2):
    recursive_gaussian_filter.SetSigma(sigma)
    recursive_gaussian_filter.Update()
    # Update the image used in the viewer
    viewer.image = recursive_gaussian_filter.GetOutput()
slider = interactive(recgauss_and_view, sigma=(0, 10, 1))

widgets.VBox([viewer, slider])

### Unsharpening Image Filter

With this filter we may use a mask, which will be used to unsharpen your image only inside the provided mask.

In [25]:
unsharp_filter = itk.UnsharpMaskImageFilter[image_itk_float, itk.Image[itk.F,3]].New(image_itk_float)
unsharp_filter.SetSigma(10)
# sobel_filter.SetZeroOrder()
unsharp_filter.Update()

unsharped_image = unsharp_filter.GetOutput()
itkw.view(unsharped_image, cmap='Grayscale', mode='x')

Viewer(cmap='Grayscale', gradient_opacity=0.22, mode='x', rendered_image=<itkImagePython.itkImageF3; proxy of …

In [26]:
itkw.checkerboard(unsharped_image, sharper_image, cmap='Grayscale', mode='x', pattern=3)